In [ ]:
import pandas as pd
import sqlite3
import json
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("whitegrid")
con = sqlite3.connect("../app.db")
df = pd.read_sql_query("SELECT * from report_histo", con)
excel = pd.read_excel("Dossier_Patient_clean_myocapt_KC_v6.xlsx")

In [ ]:
def table_to_df(df):
    tree_as_dict = {}
    features_col = []
    for index, row in df.iterrows():
        tree_as_dict.setdefault("id", []).append(row[0])
        tree_as_dict.setdefault("patient_id", []).append(row[1])
        tree_as_dict.setdefault("expert_id", []).append(row[2])
        tree_as_dict.setdefault("biopsie_id", []).append(row[3])
        tree_as_dict.setdefault("muscle_prelev", []).append(row[4])
        tree_as_dict.setdefault("age_biopsie", []).append(row[5])
        tree_as_dict.setdefault("date_envoie", []).append(row[6])
        tree_as_dict.setdefault("gene_diag", []).append(row[7])
        tree_as_dict.setdefault("comment", []).append(row[9])
        tree_as_dict.setdefault("conclusion", []).append(row[10])
        tree_as_dict.setdefault("datetime", []).append(row[11])

        my_tree = json.loads(row[8])
        for feature in my_tree:
            tree_as_dict.setdefault(feature["text"], []).append(float(feature["data"].get("presence", -0.25)))
            if index==0:
                features_col.append(feature["text"])
    return tree_as_dict, features_col
tree_as_dict, features_col = table_to_df(df)
df2 = pd.DataFrame.from_dict(tree_as_dict)
df2 = df2.replace({"COM_CCD":"COM", "COM_MMM":"COM", "NM_CAP":"NM", "CFTD":"OTHER", "NON_CM":"OTHER","CM":"UNCLEAR"})
df2 = df2.replace({-0.25:0, 0.25:1, 0.5:1, 0.75:1})


In [ ]:
muscle_prelev = df2["muscle_prelev"].value_counts()
# Empty index to N/A
as_list = muscle_prelev.index.tolist()
idx = as_list.index('')
as_list[idx] = 'N/A'
muscle_prelev.index = as_list
sns.barplot(x=muscle_prelev.index, y=muscle_prelev)
var = plt.xticks(rotation=25)

In [ ]:
age_biopsie = df2["age_biopsie"].value_counts()
bebe = age_biopsie.where(age_biopsie.index<=2).sum()
enfant = age_biopsie.where((age_biopsie.index>2) & (age_biopsie.index<18)).sum()
adulte = age_biopsie.where(age_biopsie.index>=18).sum()
g = sns.barplot(x=["Bébé (<=2ans)", "Enfant (2-18ans)", "Adulte (>=18ans)"], y=[bebe, enfant, adulte])

In [ ]:
gene_diag = df2["gene_diag"].value_counts()
fig_dims = (8, 4)
fig, ax = plt.subplots(figsize=fig_dims)
sns.barplot(x=gene_diag.index, y=gene_diag, ax=ax)
var = plt.xticks(rotation=90)

In [ ]:
# Merge sub types for stats
conclusion = df2["conclusion"].value_counts()
g = sns.barplot(x=conclusion.index, y=conclusion)

In [ ]:
df2.where(df2["conclusion"]=="NM")[features_col].sum().sort_values(ascending=False)[0:10]

In [ ]:
df2.where(df2["conclusion"]=="COM")[features_col].sum().sort_values(ascending=False)[0:10]

In [ ]:
df2.where(df2["conclusion"]=="CNM")[features_col].sum().sort_values(ascending=False)[0:10]

In [ ]:
#df2.to_csv("histo_feature.csv", index=False)